In [1]:
import numpy as np
import pandas as pd

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies=movies.merge(credits,on='title')

In [4]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [6]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [7]:
movies=movies.dropna() 

In [8]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [9]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L 

       
    

In [10]:
movies['genres']= movies['genres'].apply(convert) 

In [11]:
movies['keywords']=movies['keywords'].apply(convert)

In [12]:
import ast
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        
        if counter!=3:
            L.append(i['name'])
            counter+=1
 
        else:
            break   
        
    return L



In [13]:
movies['cast']=movies['cast'].apply(convert3)

In [14]:
import ast
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == ('Director'):
            L.append(i['name'])
            break
    return L 

In [15]:
movies['crew']=movies['crew'].apply(fetch_director)

In [16]:
 movies['overview']=movies['overview'].apply(lambda x:x.split())

In [17]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [18]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace("","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace("","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace("","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace("","") for i in x])



In [19]:
movies['tags']=movies['overview']+ movies['genres']+ movies['keywords']+ movies['cast']+ movies['crew']

In [20]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
#new.head()

In [21]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [22]:
import nltk

In [23]:
from nltk.stem.porter  import PorterStemmer
ps= PorterStemmer()


In [24]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [25]:
new['tags']=new['tags'].apply(stem)

In [26]:
new['tags']=new['tags'].apply(lambda x:x.lower())

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
    

In [28]:
vector = cv.fit_transform(new['tags']).toarray()

In [29]:
vector.shape

(4806, 5000)

In [30]:
vector[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity = cosine_similarity(vector)

In [33]:
similarity

array([[1.        , 0.06692745, 0.06554735, ..., 0.04442617, 0.03244428,
        0.        ],
       [0.06692745, 1.        , 0.04167571, ..., 0.03766218, 0.        ,
        0.01756008],
       [0.06554735, 0.04167571, 1.        , ..., 0.01844278, 0.0808122 ,
        0.        ],
       ...,
       [0.04442617, 0.03766218, 0.01844278, ..., 1.        , 0.03651484,
        0.03108349],
       [0.03244428, 0.        , 0.0808122 , ..., 0.03651484, 1.        ,
        0.06810052],
       [0.        , 0.01756008, 0.        , ..., 0.03108349, 0.06810052,
        1.        ]])

In [34]:
new[new['title'] == 'The Lego Movie'].index[0]

744

In [35]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)
        

In [36]:
recommend('Gandhi')

Lawrence of Arabia
Dil Jo Bhi Kahey...
Barry Lyndon
Guiana 1838
Gandhi, My Father


In [37]:
import pickle

In [38]:
pickle.dump(new,open('movie_list.pkl','wb'))

In [39]:
new['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [40]:
pickle.dump(new.to_dict(),open('movie_dict.pkl','wb'))

In [41]:
pickle.dump(similarity,open('similarity.pkl','wb'))